# Module 11 Challenge
## Deliverable 2: Scrape and Analyze Mars Weather Data

In [ ]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup as soup
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
browser = Browser('chrome')

### Step 1: Visit the Website

Use automated browsing to visit the [Mars Temperature Data Site](https://static.bc-edx.com/data/web/mars_facts/temperature.html). Inspect the page to identify which elements to scrape.

   > **Hint** To identify which elements to scrape, you might want to inspect the page by using Chrome DevTools to discover whether the table contains usable classes.


In [ ]:
# Visit the website
url = "https://static.bc-edx.com/data/web/mars_facts/temperature.html"
browser.visit(url)

### Step 2: Scrape the Table

Create a Beautiful Soup object and use it to scrape the data in the HTML table.

Note that this can also be achieved by using the Pandas `read_html` function. However, use Beautiful Soup here to continue sharpening your web scraping skills.

In [ ]:
# Create a Beautiful Soup Object
html = browser.html
soup = soup(html, 'html.parser')

In [ ]:
# Extract all rows of data
all_rows = soup.find_all(class_='data-row')
#all_rows

### My struggles to create a dataframe out of the Mars info on the website ###
# table = soup.find('tbody')

# one_table = table.find('tr', class_='data-row')
# print(one_table)

# # for info in one_table:
# #     print(info.text.strip())

# print('----')
# row_info = one_table.find_all('td')
# print(row_info)
# print('----')

# mars_info_list = []

# for row in row_info:
#     #print(row.text)

#     if row == row:
#         id = row.text
#         mars_dict = {"id": id
#                     #  "terrestrial_date": "",
#                     #  "sol": "",
#                     #  "ls": "",
#                     #  "month": "",
#                     #  "min_temp": "",
#                     #  "pressure": ""
#                      }
#         mars_info_list.append(mars_dict)  

# print(mars_info_list)

# # id = row_info[0]
# # print(id.text)

# # terr_date = row_info[1]
# # print(terr_date.text)

# # sol = row_info[2]
# # print(sol.text)

# # ls = row_info[3]
# # print(ls.text)

# # month = row_info[4]
# # print(month.text)

# # min_temp = row_info[5]
# # print(min_temp.text)

# # pressure = row_info[6]
# # print(pressure.text)

# # a lot of if row = 1, row =2, unti row =7.

### Step 3: Store the Data

Assemble the scraped data into a Pandas DataFrame. The columns should have the same headings as the table on the website. Here’s an explanation of the column headings:

* `id`: the identification number of a single transmission from the Curiosity rover
* `terrestrial_date`: the date on Earth
* `sol`: the number of elapsed sols (Martian days) since Curiosity landed on Mars
* `ls`: the solar longitude
* `month`: the Martian month
* `min_temp`: the minimum temperature, in Celsius, of a single Martian day (sol)
* `pressure`: The atmospheric pressure at Curiosity's location

In [ ]:
# Create an empty list
mars_temp_data_list = []

# Obtain the the table headings
table_headings = soup.find_all('th')
# Put just the text of the table headsings into a list called table_heading_list
table_heading_list=[heading.text for heading in table_headings]
# Append the table_heading_list into the mars_temp_data_list
mars_temp_data_list.append(table_heading_list)

# Loop through the scraped data to create a list of rows / Loop through all the table rows
for row in all_rows:
    # Put all the row info into a list and assign it to row_info_list
    row_info_list = [info.text for info in row.find_all('td')]
    # Append the row_info_list into the mars_temp_data_list
    mars_temp_data_list.append(row_info_list)



In [ ]:
# Create a Pandas DataFrame by using the list of rows and a list of the column names
mars_df = pd.DataFrame(mars_temp_data_list)

# Replacing DataFrame header with top row stackover flow reference: https://stackoverflow.com/questions/31328861/replacing-header-with-top-row
new_header = mars_df.iloc[0] #grab the first row for the header
mars_df = mars_df[1:] #take the data less the header row
mars_df.columns = new_header #set the header row as the df header

In [ ]:
# Confirm DataFrame was created successfully
mars_df


### Step 4: Prepare Data for Analysis

Examine the data types that are currently associated with each column. If necessary, cast (or convert) the data to the appropriate `datetime`, `int`, or `float` data types.

  > **Hint** You can use the Pandas `astype` and `to_datetime` methods to accomplish this task.


In [ ]:
# Examine data type of each column
mars_df.dtypes #all are objects in the beginning



In [ ]:
# Change data types for data analysis
# Update each column accordingly to the data type we want
mars_df = mars_df.astype({"id": int, "sol": int, "ls": int, "month": int, "min_temp": float, "pressure": float})
mars_df['terrestrial_date'] = pd.to_datetime(mars_df['terrestrial_date']) # Reference this site for this line of code https://www.geeksforgeeks.org/python-pandas-to_datetime/# 

In [ ]:
# Confirm type changes were successful by examining data types again
mars_df.dtypes

### Step 5: Analyze the Data

Analyze your dataset by using Pandas functions to answer the following questions:

1. How many months exist on Mars? [12]
2. How many Martian (and not Earth) days worth of data exist in the scraped dataset? [1867]
3. What are the coldest and the warmest months on Mars (at the location of Curiosity)? [Month listed from coldest to hottest: 3,4,2,5,1,6,12,7,11,9 and 8]
To answer this question:
    * Find the average the minimum daily temperature for all of the months.
    * Plot the results as a bar chart.
4. Which months have the lowest and the highest atmospheric pressure on Mars? [Month listed from lowest to highest pressure: 6,5,7,4,12,11,1,8,3,10,2,9]
To answer this question:
    * Find the average the daily atmospheric pressure of all the months.
    * Plot the results as a bar chart.
5. About how many terrestrial (Earth) days exist in a Martian year? [Please look at the answers for all these questions above Step 6: Save the Data]
To answer this question:
    * Consider how many days elapse on Earth in the time that Mars circles the Sun once.
    * Visually estimate the result by plotting the daily minimum temperature.


In [ ]:
# 1. How many months are there on Mars?
unique_month = mars_df['month'].unique()
unique_month

In [ ]:
# 2. How many Martian days' worth of data are there?
martian_days = mars_df['sol'].nunique() # Jesse Wright taught me about the nunique function!
martian_days

In [ ]:
# 3. What is the average low temperature by month?
average_low_temps_by_month = mars_df.groupby(['month']).mean()['min_temp']
# average_low_temps_by_month = average_low_temps_by_month
average_low_temps_by_month

In [ ]:
# Plot the average temperature by month
average_low_temps_by_month.plot.bar()
#average_low_temps_by_month.plot()



In [ ]:
# Identify the coldest and hottest months in Curiosity's location
average_low_temps_by_month = mars_df.groupby(['month']).mean()['min_temp'].sort_values()
average_low_temps_by_month.plot.bar(average_low_temps_by_month)

In [ ]:
# 4. Average pressure by Martian month
average_pressure_by_month = mars_df.groupby(['month']).mean()['pressure']
average_pressure_by_month

In [ ]:
# Plot the average pressure by month
average_pressure_by_month = mars_df.groupby(['month']).mean()['pressure'].sort_values()
average_pressure_by_month.plot.bar(average_pressure_by_month)

In [ ]:
# 5. How many terrestrial (earth) days are there in a Martian year?
mars_df.min_temp.plot()
plt.show()

On average, the third month has the coldest minimum temperature on Mars, and the eighth month is the warmest. But it is always very cold there in human terms!



Atmospheric pressure is, on average, lowest in the sixth month and highest in the ninth.

The distance from peak to peak is roughly 1425-750, or 675 days. A year on Mars appears to be about 675 days from the plot. Internet search confirms that a Mars year is equivalent to 687 earth days.

### Step 6: Save the Data

Export the DataFrame to a CSV file.

In [ ]:
# Write the data to a CSV
mars_df.to_csv("mars.csv", index=True, header=True)

In [ ]:
browser.quit()

#### Notes to self: 
- DataFrame.plot(*args, **kwargs) https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html
- plot.bar(average_low_temps_by_month) https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.bar.html


average_low_temps_by_month.plot(kind='bar')